In [ ]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# Dealing with Mysterious Conflicts

This file shows *reduce/reduce* conflicts that result from the fact that the set of LR states is compressed into the set of LALR states.

## Specification of the Scanner

We implement a minimal scanner for arithmetic expressions.

In [ ]:
import ply.lex as lex

tokens = [ 'X' ]

def t_X(t):
    r'x'
    return t

literals = ['v', 'w', 'y', 'z']

t_ignore  = ' \t'

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)

__file__ = 'main'

lexer = lex.lex()

## Specification of the Parser

In [ ]:
import ply.yacc as yacc

The *start variable* of our grammar is `expr`, but we dont't have to specify that.  The default
start variable is the first vvariable that is defined.

In [ ]:
start = 's'

We can specify multiple expressions in a single rule.  In this case, we have used the `pass` statement as we just want to check whether there are *reduce/reduce conflicts*. 

In [ ]:
def p_s(p):
    """
    s : 'v' a 'y'
      | 'w' b 'y'
      | 'v' b 'z'
      | 'w' a 'z'
      
    a : X
    
    b : X
    """
    pass

In [ ]:
def p_error(p):
    if p:
        print(f'Syntax error at {p.value}.')
    else:
        print('Syntax error at end of input.')

Setting the optional argument `write_tables` to `False` <B style="color:red">is required</B> to prevent an obscure bug where the parser generator tries  to read an empty parse table.

In [ ]:
parser = yacc.yacc(write_tables=False, debug=True)

Interestingly, there are no conflicts. If `Ply` had indeed calculated LALR tables, there would have been conflicts. **However**, `Ply` is smart enough to **not** merge states 
if this merger would result in a reduce/reduce conflict.  Below, `state 6` and `state 9`
should have been merged since they have the same core, **but** as the merger would produce
a *reduce/reduce conflict*, `Ply` does not merge these states.

In [ ]:
!cat parser.out